In [265]:
from alchemyDB import *
from create_objects import *
import sqlalchemy
from sqlalchemy import orm
import time
import pprint
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.expression import func
from sqlalchemy import inspect
import pandas as pd

engine = create_engine('sqlite:///boardgames.db')
Session = sessionmaker(bind=engine, autoflush=False)
session = Session()

In [280]:
def game_to_json(o):
    og = vars(o) 
    keys = og.keys()
    extra = ['_sa_instance_state', 'mechanics', 'categories', 'artists']
    
    d = {k: v for k, v in og.items() if k not in extra}
    d['mechanics'] = [m.name for m in o.mechanics]
    d['categories'] = [c.name for c in o.categories]
    d['artists'] = [a.name for a in o.artists]
    
    for k, v in d.items():
        if v == '-100':
            d[k] = None
    
    return d

## MVP lol

In [12]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=5)
df = pd.read_sql_query("SELECT * FROM games", engine)
relevant = ['id', 'name', 'ratingscount', 'avgrating', 'published',
       'minplayers', 'maxplayers', 'best', 'recommended', 'not_recommended',
       'playingtime', 'minplaytime', 'maxplaytime', 'minage', 'suggestedage',
       'language_dependence']
fullX = df[relevant].dropna()
X = fullX.drop(['id', 'name'], axis=1)

In [66]:
nn.fit(X.values)
import pickle

pickle.dump(nn, open('testnn.p', 'wb'))

In [67]:
loaded_model = pickle.load(open('testnn.p', 'rb'))

In [68]:
loaded_model.kneighbors(test.values.reshape(1, -1), 6)[-1]

array([[  56, 2015, 1517,  299,  841, 1770]])

In [69]:
test.values.reshape(1, -1)

array([[3.49000e+02, 5.66331e+00, 1.97400e+03, 2.00000e+00, 4.00000e+00,
        2.00000e+00, 2.00000e+00, 1.00000e+00, 1.20000e+02, 1.20000e+02,
        1.20000e+02, 1.00000e+01, 1.20000e+01, 1.00000e+00]])

In [37]:
fullX[fullX['id'] == n]

,id,name,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence
63,67,Conquest,349,5.66331,1974,2,4,2.0,2.0,1.0,120,120,120,10,12.0,1.0


In [35]:
n = 67
test = fullX[fullX['id'] == n].drop(['id', 'name'], axis=1)
test

,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence
63,349,5.66331,1974,2,4,2.0,2.0,1.0,120,120,120,10,12.0,1.0


In [36]:
nearest = nn.kneighbors(test.values.reshape(1, -1), 6)[-1]
results = nearest.tolist()[0][1:]
fullX.iloc[results,:]

,id,name,ratingscount,avgrating,published,minplayers,maxplayers,best,recommended,not_recommended,playingtime,minplaytime,maxplaytime,minage,suggestedage,language_dependence
3605,5740,Strat-O-Matic Pro Football,342,5.80789,1968,1,2,2.0,1.0,3.0,120,120,120,11,12.0,3.0
2565,3484,Armada,340,5.42925,1986,2,4,4.0,2.0,5.0,120,120,120,10,12.0,2.0
370,426,The Battle of the Bulge,366,5.71396,1981,2,2,2.0,1.0,3.0,120,120,120,12,12.0,3.0
1298,1558,Rogue Trooper,335,5.59232,1987,2,6,4.0,2.0,1.0,120,120,120,12,12.0,3.0
3088,4454,Statis Pro Football,369,5.67680,1973,1,2,2.0,1.0,3.0,120,120,120,12,12.0,2.0
